In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Abstractive Summarization


In [3]:
text_to_summarize = "Sinan Ozdemir is a Data Scientist, Entrepreneur, Teacher, and Author. He is the founder of LoopGenius, a company that helps people get their first 100 customers for their side hustle. As an expert in NLP technologies, including LLMs and ChatGPT, Sinan is passionate about exploring the power of these technologies to help businesses achieve their goals. He specializes in generative AI, including stable diffusion and GPT3, and is committed to helping companies rapidly prototype and implement these cutting-edge technologies. Sinan has authored 5 books on Machine Learning and Data Science and has taught at renowned institutions like Johns Hopkins University, O’Reilly, and Pearson."

preprocess_text = text_to_summarize.strip().replace("\n","")

preprocess_text

'Sinan Ozdemir is a Data Scientist, Entrepreneur, Teacher, and Author. He is the founder of LoopGenius, a company that helps people get their first 100 customers for their side hustle. As an expert in NLP technologies, including LLMs and ChatGPT, Sinan is passionate about exploring the power of these technologies to help businesses achieve their goals. He specializes in generative AI, including stable diffusion and GPT3, and is committed to helping companies rapidly prototype and implement these cutting-edge technologies. Sinan has authored 5 books on Machine Learning and Data Science and has taught at renowned institutions like Johns Hopkins University, O’Reilly, and Pearson.'

In [7]:
t5_prepared_text = "summarize: " + preprocess_text
input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

summary_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    min_length=30,
    max_length=50,
    early_stopping=True
)

output = base_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

output

'Sinan Ozdemir is a Data Scientist, Entrepreneur, Teacher, and Author . he is the founder of LoopGenius, a company that helps people get their first 100 customers for their side hustle .'

## English To German

In [12]:
input_ids = base_tokenizer.encode('Translate English to German: Where is the chocolate?', return_tensors="pt")

translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

output

'Wo ist die Schokolade?'

In [11]:
input_ids = base_tokenizer('Translate English to German: Where is the chocolate?', return_tensors="pt").input_ids
labels = base_tokenizer('Wo ist die Schokolade?', return_tensors="pt").input_ids

loss = base_model(input_ids=input_ids, labels=labels).loss
labels, loss

(tensor([[ 3488,   229,    67, 31267,    58,     1]]),
 tensor(0.1245, grad_fn=<NllLossBackward0>))

## CoLA: The Corpus of Linguistic Acceptability

In [15]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors="pt")

coLA_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(coLA_ids[0], skip_special_tokens=True)

output

'acceptable'

In [16]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors="pt")

coLA_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(coLA_ids[0], skip_special_tokens=True)

output

'unacceptable'

## STSB- Semantic Text Similarity Benchmark

In [18]:
s_one = "How to fish"
s_two = 'Fishing Manual for beginners'

input_ids = base_tokenizer.encode(f"stsb sentence1: {s_one} sentence2: {s_two}", return_tensors="pt")

stsb_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(stsb_ids[0], skip_special_tokens=True)

output
#Similarity is between 0-5

'3.2'

In [19]:
s_one = "How to fish"
s_two = 'Hiking Manual for beginners'

input_ids = base_tokenizer.encode(f"stsb sentence1: {s_one} sentence2: {s_two}", return_tensors="pt")

stsb_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(stsb_ids[0], skip_special_tokens=True)

output
#Similarity is between 0-5

'0.0'

## MNLI - Multi-Genre Natural Language Inference

In [20]:
input_ids = base_tokenizer.encode(
    "mnli premise: I am active in politics. hypothesis: I am running for mayor",
    return_tensors="pt")

stsb_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(stsb_ids[0], skip_special_tokens=True)

output
#Similarity is between 0-5

'entailment'

In [21]:
input_ids = base_tokenizer.encode(
    "mnli premise: I am active in politics. hypothesis: I do not really vote",
    return_tensors="pt")

stsb_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(stsb_ids[0], skip_special_tokens=True)

output
#Similarity is between 0-5

'contradiction'

In [22]:
input_ids = base_tokenizer.encode(
    "mnli premise: I am active in politics. hypothesis: I code for living",
    return_tensors="pt")

stsb_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(stsb_ids[0], skip_special_tokens=True)

output
#Similarity is between 0-5

'neutral'